Hello and welcome. In this kernel we are going to perform some exploraory data analysis some visualization and will try classify the data using Random Forest Classifier. Lets get started with loading the essential libraries.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)  
import cufflinks as cf  
cf.go_offline()  
%matplotlib inline
import os
df = pd.read_csv('../input/Dataset_spine.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

Lets start by changing the column name for better understanding and ease our work

In [ ]:
Cols = ['pelvic_incidence','pelvic_tilt','lumbar_lordosis_angle','sacral_slope','pelvic_radius','degree_spondylolisthesis','pelvic_slope',
       'Direct_tilt','thoracic_slope','cervical_tilt','sacrum_angle','scoliosis_slope']
df = df.rename(index=str,columns={'Col1':'pelvic_incidence','Col2':'pelvic_tilt','Col3':'lumbar_lordosis_angle','Col4':'sacral_slope','Col5':'pelvic_radius','Col6':'degree_spondylolisthesis','Col7':'pelvic_slope','Col8':'Direct_tilt',
                                  'Col9':'thoracic_slope','Col10':'cervical_tilt','Col11':'sacrum_angle','Col12':'scoliosis_slope'})

Firstly EDA is essential to bring out the necessary insights and inferences that the data holds

In [ ]:
df.describe()

In [ ]:
df.info()

Here we observe the varing nature of each column of the dataset

In [ ]:
for i in Cols:
    plt.figure(figsize=(20,7))
    df[i].plot(subplots=True)
    plt.title(i)
    plt.show()

Lets observe the classes which will be needed for classification

In [ ]:
A = df['Class_att'].value_counts().index.tolist()
Vals = list(pd.value_counts(df['Class_att']))
explode = [0.1]*len(A)
plt.figure(figsize=(10,10))
plt.pie(Vals,explode=explode,labels=A,autopct='%.1f%%')
plt.show()

In [ ]:
sns.pairplot(df)

In [ ]:
sns.pairplot(hue='Class_att',data=df)

In [ ]:
df.iplot(kind='box')  

For Classification we are considering to use Random Forest Classifier

In [ ]:
from sklearn.preprocessing import LabelEncoder,scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,roc_curve,roc_auc_score,confusion_matrix
from sklearn.model_selection import train_test_split as t

Firstly we separate the target variable

In [ ]:
x = df.iloc[:,:-1].values 
y = df.iloc[:,-1].values  #Target Variable

And Label Encode the target variable since the model only accepts the integer values

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

Keeping Train-Test Splitting of the dataset we decided to go ahead with training as 80% and testing as 20%

In [ ]:
train_x,test_x,train_y,test_y = t(x,y,test_size=0.2)

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting (source: Documentation)
We taken 30 estimators which means that we will have 30 trees in total in our forest, max depth of 2.

In [ ]:
rfc = RandomForestClassifier(n_estimators=30,max_depth=2)
rfc.fit(train_x, train_y)
score_randomforest = rfc.score(test_x,test_y)
print('The accuracy of the Random Forest Model is', score_randomforest)

Here we have the Confusion Matrix of the above Classification

In [ ]:
df_cm  = pd.DataFrame(confusion_matrix(test_y,rfc.predict(test_x)),['Abnormal','Normal'],['Abormal','Normal'])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

Here is the Classification Report

In [ ]:
print(classification_report(test_y,rfc.predict(test_x)))

And here is the ROC Curve for the model

In [ ]:
probs = rfc.predict_proba(test_x)
probs = probs[:,1]
auc = roc_auc_score(test_y,probs)
print('AUC = %.3f' % auc)
fpr,tpr,thresholds = roc_curve(test_y,probs)
plt.plot([0,1],[0,1],linestyle='--')
plt.plot(fpr,tpr,marker='.')
plt.show()